In [1]:
#CONNEXION à google drive
import pandas as pd
from google.colab import drive

drive.mount('/Drive')

Mounted at /Drive


In [6]:
#PARAMETRES
pathSaveCsv = '/Drive/My Drive'

In [2]:
#CHARGEMENT des fichiers
import re  
import requests
import io

url = "https://raw.githubusercontent.com/JulienJ-44/rakuteam/main/Datasets/Dataset_challenge.csv"
download = requests.get(url).content
X=pd.read_csv(io.StringIO(download.decode('utf-8')))


#y = pd.read_csv("/Drive/My Drive/Colab/Y_train_CVw08PX.csv", index_col=0) 

#stockage de tous les codes de classe dans une liste
#classes_codes = (y['prdtypecode'].value_counts().index.tolist())

#PREPARATION: FUSION de X et y et remplacement des VALEURS MANQUANTES par ""
df = X.drop(['imageid'], axis = 1)#
#on joint X et y
#df = pd.merge(df1, y, left_index=True, right_index=True)

#valeurs MANQUANTES
df['description']=df['description'].fillna("")
df['designation']=df['designation'].fillna("")
#turn all text to LOWER case
df['description'] = df['description'].str.lower()
df['designation'] = df['designation'].str.lower()

#recodage des caractères HTML
df.replace(["&eacute;","&egrave;","&euml;","&ecirc;","&ocirc;","&ugrave;","&ccedil;","&agrave;","&nbsp;","&amp;","&#39;","'",'n°']
           , ["é","è","ë","ê","ô","ù","ç","à"," ","&"," "," ",'numéro '], inplace=True, regex=True)

"""
Return the text in lower case after removing html tags
"""    
def pre_process(text):
    # lowercase
    #text=text.lower()
    #remove tags
    text=re.sub('<.*?>'," ",text) 
    return text

df['description'] = df['description'].apply(lambda x:pre_process(x))
df['designation'] = df['designation'].apply(lambda x:pre_process(x))

In [3]:
#FONCTION pour supprimer les mots inutiles
# Download the stopwords
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk as nltk
nltk.download('punkt') #télécharge les paquets language (dont FR)
nltk.download('stopwords')

#Ajoute des stopwords à la liste française prédéfinie
stop_words = set(stopwords.words('french'))
stop_words.update(stopwords.words('english'))
stop_words.update(stopwords.words('german'))
stop_words.update(['-','de','en','pour','la','(',')','the','/li','/strong','strong','and','non','eacute','comme','cette','of'])

def filtrer_mots_inutiles(texte):
  """
  Fonction pour transformer un texte en une suite de mots séparés par des espaces
  et en excluant les stopwords et les mots de moins de 2 caractères
  """
  mots = word_tokenize(texte, language='french')
  tokens = []
  for mot in mots:
    if (mot not in stop_words and len(mot)>1):
      tokens.append(mot.lower())
  mastring=" ".join(tokens)
  return mastring

df['designation']= df['designation'].apply(lambda x: filtrer_mots_inutiles(x))
df['description']= df['description'].apply(lambda x: filtrer_mots_inutiles(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
df.to_csv(f'{pathSaveCsv}/dataset_challenge_cleaned.csv')

In [7]:

df=pd.read_csv(f'{pathSaveCsv}/dataset_challenge_cleaned.csv')
df['designation']=df['designation'].fillna("")
#valeurs MANQUANTES
df['description']=df['description'].fillna("")

In [8]:
#Tokenisation (Séparation du texte en phrases et mots) du champ "designation"

#en phrases
tokenizer = PunktSentenceTokenizer()
df['desi_nb_phrases']= df['designation'].apply(lambda x: len(tokenizer.tokenize(str(x))))
df['desc_nb_phrases']= df['description'].apply(lambda x: len(tokenizer.tokenize(str(x))))
#en mots
#df['desi_nb_mots+']= df['designation'].apply(lambda x: len(x.split(" "))
#df['desc_nb_mots+']= df['description'].apply(lambda x: len(x.split(" "))

In [ ]:
#IDF ("inverse de fréquence de document")
#formule: log(nbr d articles / nb d articles dans lequels le terme apparaît)
from sklearn.feature_extraction.text import TfidfVectorizer
#transformation de la colonne en type liste
words_desi=df['designation'].tolist()
#words_desc=df['description'].tolist()

# create object 
vect  = TfidfVectorizer()  
# get tf-df values 
result = vect.fit_transform(words_desi) 
# get idf values 
#sorted_vocabulary = dict(sorted(vect.vocabulary_.items(), key=lambda item: item[0], reverse=True))
print(vect.vocabulary_)
print(vect.get_feature_names())
print(vect.idf_)

{'olivia': 62494, 'personalisiertes': 65414, 'notizbuch': 61528, '150': 2294, 'seiten': 75389, 'punktraster': 69046, 'ca': 23158, 'din': 32742, 'a5': 12875, 'rosen': 72508, 'design': 32114, 'journal': 49932, 'arts': 16751, '133': 1799, '28': 5480, '09': 432, '2001': 3898, 'art': 16651, 'marche': 56049, 'salon': 73860, 'asiatique': 16834, 'paris': 64418, 'jacques': 49195, 'barrere': 18788, 'francois': 40529, 'perrier': 65379, 'reforme': 70810, 'ventes': 85672, 'encheres': 36341, 'publiques': 68942, 'sna': 77060, 'fete': 38887, 'cent': 25298, 'ans': 15431, 'grand': 43317, 'stylet': 78929, 'ergonomique': 36966, 'bleu': 20423, 'gamepad': 41650, 'nintendo': 61187, 'wii': 87829, 'speedlink': 77881, 'pilot': 66031, 'style': 78927, 'peluche': 65097, 'donald': 33543, 'europe': 37479, 'disneyland': 32952, '2000': 3879, 'marionnette': 56161, 'doigt': 33451, 'guerre': 43978, 'tuques': 84041, 'afrique': 13801, 'contemporaine': 28594, '212': 4273, 'hiver': 45782, '2004': 3913, 'dossier': 33711, 'jap

In [ ]:
#fonctions de manipulation des TFIDF
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [ ]:
#TF fréquence du terme dans l ensbl de la categorie d articles
feature_names=vect.get_feature_names()
#on étudie le champ designation ( à remplacer par "description" si besoin)
colonneClass = "designation" 
class_text_cols = []
#on parcourt la liste des codes de classes
lst_keywords_byclass={}
classes_codes = (df['prdtypecode'].value_counts().index.tolist())
#classes_codes=[2583, 1220, 22240, ...]
for class_code in classes_codes:
  #on copie dans un df les lignes correspondant au code de classe actuel
  df_class = df[df["prdtypecode"]==class_code]
  #on joint tous les élements de la colonne dans une chaîne
  class_text = ' '.join(df_class[colonneClass])
  #affichage des infos de la classe
  print("Classe:", class_code, "\n", len(df_class.index), " éléments")
  #tfidf
  class_text_encode = vect.transform([class_text])

  
  #sort the tf-idf vectors by descending order of scores
  sorted_items=sort_coo(class_text_encode.tocoo())
  #extract only the top n; n here is 10
  keywords=extract_topn_from_vector(feature_names,sorted_items,15)
  # now print the results
  print("===Keywords===")
  for k in keywords:
      print(k,keywords[k])
  lst_keywords_byclass[class_code]=keywords
  print("\n")

#lst_keywords_byclass contient: {1180:[mot1 mot2 mot3 ...], 2520:[mot1 mot2 mot3 ...] ...}

Classe: 2583.0 
 10209  éléments
===Keywords===
piscine 0.58
kit 0.288
bâche 0.243
spa 0.194
pompe 0.161
bulles 0.154
intex 0.148
cm 0.137
ronde 0.123
acier 0.12
gonflable 0.12
filtration 0.111
gré 0.111
filtre 0.106
bois 0.106


Classe: 1560.0 
 5073  éléments
===Keywords===
cm 0.271
matelas 0.224
cuisine 0.222
table 0.172
chaise 0.164
blanc 0.157
bois 0.15
tissu 0.146
rangement 0.141
salle 0.128
fournitures 0.123
design 0.115
sommier 0.115
gris 0.114
pieds 0.113


Classe: 1300.0 
 5045  éléments
===Keywords===
générique 0.603
rc 0.338
drone 0.312
dji 0.241
quadcopter 0.157
mavic 0.143
avion 0.128
fpv 0.127
caméra 0.119
maquette 0.118
voiture 0.112
wifi 0.108
car 0.089
pro 0.089
batterie 0.085


Classe: 2060.0 
 4993  éléments
===Keywords===
led 0.293
noël 0.26
décoration 0.229
lampe 0.228
décor 0.204
bricolage 0.171
peinture 0.161
lumière 0.156
diamant 0.138
broderie 0.124
mur 0.124
5d 0.122
chambre 0.118
bain 0.118
3d 0.117


Classe: 2522.0 
 4989  éléments
===Keywords===
notes 0.47

In [ ]:
#TEMP df_sample!!!
#df_sample=df.head(1000) #TEMP!!!!
#la liste des keywords stockés par classe a été limitée à 10, faut il aller plus loin, performances?
"""
Fontion de scoring qui renvoie un score selon le nombre d'occurences des mots et leur score idf
texte: texte à lire mot à mot pour scorer les mots
dict_keywords_idf:dictionnaire des keywords/idf d'une des classes produits
"""
import re
def scoring(texte, dict_keywords_idf):
#texte = texte de la désignaton/description de l'article
#dict_keywords_idf = [mot1 mot2 mot3 ...]
  #print(texte)
  #print(dict_keywords_idf)
  score = 0.0
  for keyword_key,keyword_idf in dict_keywords_idf.items():
    nb_occur = len(re.findall(keyword_key, texte, flags=re.IGNORECASE))
    score += nb_occur*keyword_idf
  #print(score)
  return score

df["best_idf"]=0
#lst_keywords_byclass contient: {1180:[mot1 mot2 mot3 ...], 2520:[mot1 mot2 mot3 ...] ...}
#lst_keywords_byclass = sorted(lst_keywords_byclass)
for (key_kw, value_kw) in sorted(lst_keywords_byclass.items()):
  #key_kw=1180
  #value_kw=[mot1 mot2 mot3 ...]
  df['class_'+str(key_kw)]=df['designation'].apply(lambda x: scoring(x, value_kw))

In [ ]:
df = df.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)
df.head()

,designation,description,productid,prdtypecode,desi_nb_phrases,desc_nb_phrases,best_idf,class_10.0,class_40.0,class_50.0,class_60.0,class_1140.0,class_1160.0,class_1180.0,class_1280.0,class_1281.0,class_1300.0,class_1301.0,class_1302.0,class_1320.0,class_1560.0,class_1920.0,class_1940.0,class_2060.0,class_2220.0,class_2280.0,class_2403.0,class_2462.0,class_2522.0,class_2582.0,class_2583.0,class_2585.0,class_2705.0,class_2905.0
0,olivia personalisiertes notizbuch 150 seiten p...,,3804725264,10.0,1,0,0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.0,0.157,0.115,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.190,0.0,0.0,0.115,0.117,0.021
1,journal arts n° 133 28/09/2001 l'art marche sa...,,436067568,2280.0,1,0,0,0.306,0.0,0.00,0.0,0.0,0.131,0.0,0.176,0.000,0.338,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.834,0.000,0.000,0.272,0.0,0.0,0.000,0.000,0.000
2,grand stylet ergonomique bleu gamepad nintendo...,pilot style touch pen marque speedlink stylet ...,201115110,50.0,1,1,0,0.000,0.1,0.48,0.0,0.0,0.000,0.0,0.000,0.275,0.000,0.0,0.0,0.144,0.000,0.0,0.26,0.0,0.0,0.000,0.643,0.657,0.297,0.0,0.0,0.000,0.000,0.020
3,peluche donald europe disneyland 2000 marionne...,,50418756,1280.0,1,0,0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.222,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.000
4,guerre tuques,luc idées grandeur veut organiser jeu guerre b...,278535884,2705.0,1,1,0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.109,0.000


In [ ]:
col_start=7
df["best_idf"]=df.iloc[:,col_start:].idxmax(axis=1)
#df["best_idf"]= [2583 if (s.sum()==0) else df.iloc[:,col_start:].idxmax(axis=1) for s in df.iloc[:,1col_start1:]]
#gestion des cas où toutes les colonnes valent 0
#on affecte la classe 2583 (majoritaire)
for i in range(len(df)):
  sum = df.iloc[i,col_start:].sum()
  if (sum == 0):
    df["best_idf"][i]="2583"

print("#Articles:", len(df))
print(df["best_idf"].value_counts())
#path = '/Drive/My Drive/Projet Rakuten'
#df.to_csv(f'{path}/damien.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


#Articles: 98728
class_2583.0    8450
class_1300.0    7275
class_1160.0    6987
class_2280.0    6863
2583            6060
class_2403.0    5999
class_2060.0    4525
class_1920.0    4501
class_1560.0    4214
class_2582.0    3992
class_1320.0    3945
class_2522.0    3350
class_1280.0    3279
class_2705.0    3148
class_2905.0    2746
class_1140.0    2595
class_60.0      2507
class_40.0      2375
class_2585.0    2090
class_1940.0    2086
class_2220.0    2080
class_10.0      1694
class_1281.0    1527
class_1301.0    1502
class_50.0      1490
class_1302.0    1437
class_2462.0    1244
class_1180.0     767
Name: best_idf, dtype: int64


In [ ]:
df.head()

,designation,description,productid,prdtypecode,desi_nb_phrases,desc_nb_phrases,best_idf,class_10.0,class_40.0,class_50.0,class_60.0,class_1140.0,class_1160.0,class_1180.0,class_1280.0,class_1281.0,class_1300.0,class_1301.0,class_1302.0,class_1320.0,class_1560.0,class_1920.0,class_1940.0,class_2060.0,class_2220.0,class_2280.0,class_2403.0,class_2462.0,class_2522.0,class_2582.0,class_2583.0,class_2585.0,class_2705.0,class_2905.0
0,olivia personalisiertes notizbuch 150 seiten p...,,3804725264,10.0,1,0,class_2522.0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.0,0.157,0.115,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.190,0.0,0.0,0.115,0.117,0.021
1,journal arts n° 133 28/09/2001 l'art marche sa...,,436067568,2280.0,1,0,class_2280.0,0.306,0.0,0.00,0.0,0.0,0.131,0.0,0.176,0.000,0.338,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.834,0.000,0.000,0.272,0.0,0.0,0.000,0.000,0.000
2,grand stylet ergonomique bleu gamepad nintendo...,pilot style touch pen marque speedlink stylet ...,201115110,50.0,1,1,class_2462.0,0.000,0.1,0.48,0.0,0.0,0.000,0.0,0.000,0.275,0.000,0.0,0.0,0.144,0.000,0.0,0.26,0.0,0.0,0.000,0.643,0.657,0.297,0.0,0.0,0.000,0.000,0.020
3,peluche donald europe disneyland 2000 marionne...,,50418756,1280.0,1,0,class_1280.0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.222,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.000
4,guerre tuques,luc idées grandeur veut organiser jeu guerre b...,278535884,2705.0,1,1,class_2705.0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.109,0.000


In [9]:
#Expressions régulières pour identifier les différentes unités
#TODO transformer en numérique pas en liste
import re

#nombre de nombres à 2 chiffres ou +
r = re.compile("[0-9]{2,}") 
#df['desi_nb2chiffres+']= df['designation'].apply(lambda x: min(1,len(r.findall(x))))
df['desc_nb2chiffres+']= df['description'].apply(lambda x: min(1,len(r.findall(x))))
#XGo ou XMo ou XTo
#r = re.compile("([\d.]+)\s?(go|mo|to|Go|Mo|To|giga|gigas)") 
#df['desi_Go']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
#df['desc_Go']=df['description'].apply(lambda x: min(1,len(r.findall(x))))
#N°X
r = re.compile("(numéro )") 
#df['desi_num']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
df['desi_num']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
r = re.compile("[Nn][°]\s?[\d]+") 
#df['desi_num']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
df['desc_num']=df['description'].apply(lambda x: min(1,len(r.findall(x))))
#Poids kg Kg mg g
#r = re.compile("[0-9\.]+[kKm]?[g]") #([\d.]+)\s+(lbs?|oz|g|kg) 
r = re.compile("([\d.]+)\s?(KG|Kg|g|kg|mg)[\s.]") #(nombres ou .)+ / (espace)+ / (kg mg ..)
#df['desi_poids']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
df['desc_poids']=df['description'].apply(lambda x: min(1,len(r.findall(x))))
#Taille cm mm m
r = re.compile("([\d.]+)\s?(cm|mm|m|M)[\s.]") 
#df['desi_long']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
df['desc_long']=df['description'].apply(lambda x: min(1,len(r.findall(x))))
#Volume mL cL dL
r = re.compile("([\d.]+)\s?(mL|L|ml|l|cl)[\s.]") 
#df['desi_vol']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
df['desc_vol']=df['description'].apply(lambda x: min(1,len(r.findall(x))))
#Age
r = re.compile("([\d.]+)\s?(an|ans|An|Ans|mois|Mois)[\s.]") 
#df['desi_ans_mois']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
df['desc_ans_mois']=df['description'].apply(lambda x: min(1,len(r.findall(x))))
#Pièces
r = re.compile("([\d.]+)\s?(pc|pcs|pièces|pièce)[\s.]") 
#df['desi_pieces']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
df['desc_pieces']=df['description'].apply(lambda x: min(1,len(r.findall(x))))
#%
#r = re.compile("([\d.]+)\s?%") 
#df['desi_pourcent']=df['designation'].apply(lambda x: min(1,len(r.findall(x))))
#df['desc_pourcent']=df['description'].apply(lambda x: min(1,len(r.findall(x))))

In [ ]:
import re
def feature(df) :
    df['desi_word_count'] = df['designation'].apply(lambda x : len(str(x).split()))
    df['desi_char_count (w/o space)'] = df['designation'].apply(lambda x : len(x.replace(" ","")))
    #df['desi_word_density'] = df['desi_word_count'] / (df['desi_char_count'] + 1)
    df['desi_total_length'] = df['designation'].apply(len)
    #df['desi_capitals'] = df['designation'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
    # df['desi_caps_vs_length'] = df.apply(lambda row: float(row['desi_capitals'])/float(row['desi_total_length']),axis=1)
    df['desi_num_exclamation_marks'] =df['designation'].apply(lambda x: x.count('!'))
    df['desi_num_question_marks'] = df['designation'].apply(lambda x: x.count('?'))
    #df['desi_num_punctuation'] = df['designation'].apply(lambda x: sum(x.count(w) for w in '.,;:'))
    #df['desi_num_symbols'] = df['designation'].apply(lambda x: sum(str(x).count(w) for w in '*&$%+-/'))
    df['desi_num_unique_words'] = df['designation'].apply(lambda x: len(set(w for w in x.split())))
    df['desi_words_vs_unique'] = df['desi_num_unique_words'] / df['desi_word_count']
    df["desi_word_unique_percent"] =  df["desi_num_unique_words"]*100/df['desi_word_count']
    
    df['descri_word_count'] = df['description'].apply(lambda x : len(str(x).split()))
    df['descri_char_count (w/o space)'] = df['description'].apply(lambda x : len(str(x).replace(" ","")))
    #df['descri_word_density'] = df['descri_word_count'] / (df['descri_char_count'] + 1)
    df['descri_total_length'] = df['description'].apply(lambda x :len(str(x)))
   # df['descri_capitals'] = df['description'].apply(lambda comment: sum(1 for c in str(comment) if c.isupper()))
    # df['descri_caps_vs_length'] = df.apply(lambda row: float(row['descri_capitals'])/float(row['descri_total_length']),axis=1)
    df['descri_num_exclamation_marks'] =df['description'].apply(lambda x: str(x).count('!'))
    df['descri_num_question_marks'] = df['description'].apply(lambda x: str(x).count('?'))
    #df['descri_num_punctuation'] = df['description'].apply(lambda x: sum(str(x).count(w) for w in '.,;:'))
   # df['descri_num_symbols'] = df['description'].apply(lambda x: sum(str(x).count(w) for w in '*&$%'))
    df['descri_num_unique_words'] = df['description'].apply(lambda x: len(set(w for w in str(x).split())))
    df['descri_words_vs_unique'] = df['descri_num_unique_words'] / df['descri_word_count']
    df["descri_word_unique_percent"] =  df["descri_num_unique_words"]*100/df['descri_word_count']
    return df

In [10]:
df.head()

,Unnamed: 0,Unnamed: 0.1,designation,description,productid,prdtypecode,desi_nb_phrases,desc_nb_phrases,desc_nb2chiffres+,desi_num,desc_num,desc_poids,desc_long,desc_vol,desc_ans_mois,desc_pieces
0,0,0,olivia personalisiertes notizbuch 150 seiten p...,,3804725264,10.0,1,0,0,0,0,0,0,0,0,0
1,1,1,journal arts numéro 133 28/09/2001 art marche ...,,436067568,2280.0,1,0,0,1,0,0,0,0,0,0
2,2,2,grand stylet ergonomique bleu gamepad nintendo...,pilot style touch pen marque speedlink stylet ...,201115110,50.0,1,1,1,0,0,0,0,0,0,0
3,3,3,peluche donald europe disneyland 2000 marionne...,,50418756,1280.0,1,0,0,0,0,0,0,0,0,0
4,4,4,guerre tuques,luc idées grandeur veut organiser jeu guerre b...,278535884,2705.0,1,1,0,0,0,0,0,0,0,0
